In [1]:
import sys
sys.path.append("/home/ubuntu/roaddetection/")

In [2]:
#%load_ext autoreload
#%autoreload 2

from keras.layers import merge
from src.models.data import *
from src.models.model import *

from src.data.utils import get_tile_prefix
#import rasterio.plot as rioplot
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

Using TensorFlow backend.


In [3]:
import os, shutil

original_dataset_dir = "../../data/train"
raw_images_path = "../../data/raw/images"
dirs = []

base_dir = "../../data"

train_dir = os.path.join(base_dir, "train")
dirs.append(train_dir)
validation_dir = os.path.join(base_dir, "validate")
dirs.append(validation_dir)
test_dir = os.path.join(base_dir, "test")
dirs.append(test_dir)

In [4]:
for directory in dirs:
    if not os.path.exists(directory):
       os.mkdir(directory)
    

In [5]:
from pathlib import Path


In [7]:
def should_make_tiles_from(r_analytic_name):
    is_analytic_tif = r_analytic_name.endswith(
        ('AnalyticMS.tif', 'AnalyticMS_SR.tif', 'AnalyticMS.tiff', 'AnalyticMS_SR.tiff')
    )
    return is_analytic_tif 

In [6]:
file_prefixes = [ get_tile_prefix(r_analytic.name) 
                  for r_analytic in Path(raw_images_path).iterdir()  
                    if  should_make_tiles_from(r_analytic.name)
                ]
print(file_prefixes)

NameError: name 'should_make_tiles_from' is not defined

In [11]:
validation_fnames = ["{0:s}_{1:04d}.tif".format(prefix, i) for i in range(1,10) for prefix in file_prefixes]
for fname in validation_fnames:
    for file_type in ["sat", "map", "sat_rgb"]:
        src = os.path.join(train_dir, file_type, fname)
        dest = os.path.join(validation_dir, file_type, fname)
        shutil.move(src, dest)

In [14]:
test_fnames = ["{0:s}_{1:04d}.tif".format(prefix, i) for i in range(50,60) for prefix in file_prefixes]
for fname in test_fnames:
    for file_type in ["sat", "map", "sat_rgb"]:
        src = os.path.join(train_dir, file_type, fname)
        dest = os.path.join(test_dir, file_type, fname)
        shutil.move(src, dest)

In [8]:
for directory in dirs:
    for file_type in ["sat", "map", "sat_rgb"]:
        target = os.path.join(directory, file_type)
        print(target, ":", len(os.listdir(target)))
    
print("Done.")

../../data/train/sat : 2715
../../data/train/map : 2715
../../data/train/sat_rgb : 2715
../../data/validate/sat : 171
../../data/validate/map : 171
../../data/validate/sat_rgb : 171
../../data/test/sat : 380
../../data/test/map : 380
../../data/test/sat_rgb : 380
Done.


In [9]:
data_gen_args = dict(
                    data_format="channels_last",
                    horizontal_flip=True
 )
train_gen = trainGenerator(5,'../../data/train','sat','map',
                        data_gen_args,save_to_dir = None,image_color_mode="rgba", target_size=(512,512))

validation_gen = trainGenerator(5,'../../data/validation','sat','map',
                        data_gen_args,save_to_dir = None,image_color_mode="rgba", target_size=(512,512))

In [10]:
model = unet(pretrained_weights="../../models/unet_membrane_analytic_27_08_14_55.hdf5")
model_checkpoint = ModelCheckpoint('../../models/unet_membrane_analytic_31_08_22_00.hdf5', monitor='loss',verbose=1, save_best_only=True)
history = model.fit_generator(train_gen,
                              steps_per_epoch=100,
                              epochs=25,
                              callbacks=[model_checkpoint],
                              validation_data=validation_gen,
                              validation_steps=50
                             )

/home/ubuntu/roaddetection/src/models/model.py:60: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=inputs, output=conv10)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 4)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 2368        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 256, 256, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

Epoch 1/25
Found 0 images belonging to 1 classes.Found 2715 images belonging to 1 classes.

Found 0 images belonging to 1 classes.
Found 2715 images belonging to 1 classes.
 83/100 [=======================>......] - ETA: 1:01 - loss: 0.0562 - acc: 0.9897

KeyboardInterrupt: 